<font size = "13px;">Kaggle Competition - [MLB Forecasting](https://www.kaggle.com/c/mlb-player-digital-engagement-forecasting/data)</font>

**Binary columns** will have null values as well as zeroes. <br>
**Zeroes** will occur if a player had an opportunity to do something, but did not.<br>
**Nulls** will occur if a player never had the opportunity to do something (for example: a player who does not pitch on a given day cannot possibly pitch a shutout - therefore a null value is expected)

# Data Loading

In [ ]:
import sys
import csv

csv.field_size_limit(100000000)

In [19]:
from google.colab import drive
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

drive.mount('/content/gdrive')

filenames = ['teams', 'seasons', 'players', 'awards']
path = "/content/gdrive/My Drive/Colab Notebooks/MLB/"

for x in range(0, 6):
    globals()['train%s' % x] = pd.read_csv(path + "train_%s.csv" % x, engine = 'python', error_bad_lines=False)

for filename in filenames:
    globals()['%s' % filename] = pd.read_csv(path + "%s.csv" % filename, engine = 'python', error_bad_lines=False)

example_test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/MLB/example_test.csv')
example_sample_submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/MLB/example_sample_submission.csv')

train = pd.concat([train0, train1, train2, train3, train4, train5], axis = 0)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
import ipywidgets as widgets

kaggle_data_tabs = widgets.Tab()
kaggle_data_tabs.children = list([widgets.Output() for df_name in df_names])

for index in range(0, len(df_names)):
    
    kaggle_data_tabs.set_title(index, df_names[index])
    
    with kaggle_data_tabs.children[index]:
        display(eval(df_names[index]))

display(kaggle_data_tabs)

# EDA

# Preprocessing

In [13]:
train.isna().sum()

Unnamed: 0                    0
date                          0
nextDayPlayerEngagement       0
games                       577
rosters                       0
playerBoxScores             678
teamBoxScores               678
transactions                112
standings                   685
awards                      919
events                      680
playerTwitterFollowers     1172
teamTwitterFollowers       1172
dtype: int64

## JSON Parsing (reffering to [this template](https://www.kaggle.com/ruriarmandhani/mlb-forecasting-ann))

In [6]:
import tqdm
from datetime import timedelta

def json_to_df(df, column):
    num_rows = len(df)
    
    data_list = []
    for row in tqdm.tqdm(range(num_rows)):
        
        json_data = df.iloc[row][column]
        if str(json_data) != "nan":
            data = pd.read_json(json_data)
            data_list.append(data)
        
    all_data = pd.concat(data_list, axis = 0)
    
    return all_data

A 'player_engagement' column contatins 2018-01-01 ~ 2021-04-26 data.

In [7]:
player_engagement = json_to_df(train, 'nextDayPlayerEngagement')
player_engagement.insert(0, 'date', pd.to_datetime(player_engagement['engagementMetricsDate'])-\
                                                   timedelta(days=1))
player_engagement['engagementMetricsDate'] = pd.to_datetime(player_engagement['engagementMetricsDate'])
player_engagement.reset_index(drop=True, inplace=True)
print(player_engagement.shape)
player_engagement.head()

100%|██████████| 1212/1212 [00:24<00:00, 49.42it/s]


(2497932, 7)


,date,engagementMetricsDate,playerId,target1,target2,target3,target4
0,2018-01-01,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294
1,2018-01-01,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118
2,2018-01-01,2018-01-02,519317,0.974327,56.177043,13.693746,64.166667
3,2018-01-01,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745
4,2018-01-01,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373


In [9]:
player_engagement

,date,engagementMetricsDate,playerId,target1,target2,target3,target4
0,2018-01-01,2018-01-02,628317,0.011167,4.474708,0.005168,5.735294
1,2018-01-01,2018-01-02,547989,0.042993,5.593385,0.045033,2.794118
2,2018-01-01,2018-01-02,519317,0.974327,56.177043,13.693746,64.166667
3,2018-01-01,2018-01-02,607625,0.006700,2.675097,0.005168,1.862745
4,2018-01-01,2018-01-02,592547,0.001117,0.632296,0.002953,0.931373
...,...,...,...,...,...,...,...
2497927,2021-04-26,2021-04-27,642100,0.000000,0.021540,0.000000,0.194422
2497928,2021-04-26,2021-04-27,571679,0.000324,0.018463,0.000000,0.100563
2497929,2021-04-26,2021-04-27,664199,0.001785,0.166169,0.000000,0.583266
2497930,2021-04-26,2021-04-27,595453,0.008112,0.547743,0.000000,0.415661


In [8]:
train

,Unnamed: 0,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,0,20180101,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,1,20180102,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,2,20180103,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,3,20180104,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,4,20180105,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,1207,20210422,"[{""engagementMetricsDate"":""2021-04-23"",""player...","[{""gamePk"":634508,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-22"",""t...","[{""home"":1,""gamePk"":634419,""gameDate"":""2021-04...","[{""home"":1,""teamId"":137,""gamePk"":634432,""gameD...","[{""transactionId"":479064,""playerId"":null,""play...","[{""season"":2021,""gameDate"":""2021-04-22"",""divis...",NaN,"[{""gamePk"":634436,""gameDate"":""2021-04-22"",""gam...",NaN,NaN
198,1208,20210423,"[{""engagementMetricsDate"":""2021-04-24"",""player...","[{""gamePk"":634369,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-23"",""t...","[{""home"":0,""gamePk"":634413,""gameDate"":""2021-04...","[{""home"":0,""teamId"":134,""gamePk"":634434,""gameD...","[{""transactionId"":479236,""playerId"":461872,""pl...","[{""season"":2021,""gameDate"":""2021-04-23"",""divis...",NaN,"[{""gamePk"":634378,""gameDate"":""2021-04-23"",""gam...",NaN,NaN
199,1209,20210424,"[{""engagementMetricsDate"":""2021-04-25"",""player...","[{""gamePk"":634358,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-24"",""t...","[{""home"":0,""gamePk"":634358,""gameDate"":""2021-04...","[{""home"":1,""teamId"":138,""gamePk"":634395,""gameD...","[{""transactionId"":479350,""playerId"":608686,""pl...","[{""season"":2021,""gameDate"":""2021-04-24"",""divis...",NaN,"[{""gamePk"":634361,""gameDate"":""2021-04-24"",""gam...",NaN,NaN
200,1210,20210425,"[{""engagementMetricsDate"":""2021-04-26"",""player...","[{""gamePk"":634344,""gameType"":""R"",""season"":2021...","[{""playerId"":408234,""gameDate"":""2021-04-25"",""t...","[{""home"":1,""gamePk"":634344,""gameDate"":""2021-04...","[{""home"":0,""teamId"":141,""gamePk"":634376,""gameD...","[{""transactionId"":479358,""playerId"":642092,""pl...","[{""season"":2021,""gameDate"":""2021-04-25"",""divis...","[{""awardId"":""NLPOW"",""awardName"":""NL Player of ...","[{""gamePk"":634376,""gameDate"":""2021-04-25"",""gam...",NaN,NaN


# Modeling

# Evaluation

[Nice template](https://www.kaggle.com/ryanholbrook/getting-started-with-mlb-player-digital-engagement)

In [14]:
import mlb
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['target1'] = 100 #make predictions here
    env.predict(sample_prediction_df)

,date,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20210426,"[{""gamePk"":634374,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-26"",""t...","[{""home"":1,""gamePk"":634377,""gameDate"":""2021-04...","[{""home"":1,""teamId"":139,""gamePk"":634343,""gameD...","[{""transactionId"":480386,""playerId"":543685,""pl...","[{""season"":2021,""gameDate"":""2021-04-26"",""divis...",NaN,"[{""gamePk"":634433,""gameDate"":""2021-04-26"",""gam...",NaN,NaN
1,20210427,"[{""gamePk"":634318,""gameType"":""R"",""season"":2021...","[{""playerId"":443558,""gameDate"":""2021-04-27"",""t...","[{""home"":1,""gamePk"":634320,""gameDate"":""2021-04...","[{""home"":1,""teamId"":117,""gamePk"":634333,""gameD...","[{""transactionId"":480456,""playerId"":642162,""pl...","[{""season"":2021,""gameDate"":""2021-04-27"",""divis...",NaN,"[{""gamePk"":634332,""gameDate"":""2021-04-27"",""gam...",NaN,NaN
2,20210428,"[{""gamePk"":634309,""gameType"":""R"",""season"":2021...","[{""playerId"":429722,""gameDate"":""2021-04-28"",""t...","[{""home"":1,""gamePk"":634310,""gameDate"":""2021-04...","[{""home"":0,""teamId"":111,""gamePk"":634310,""gameD...","[{""transactionId"":480728,""playerId"":545358,""pl...","[{""season"":2021,""gameDate"":""2021-04-28"",""divis...",NaN,"[{""gamePk"":634317,""gameDate"":""2021-04-28"",""gam...",NaN,NaN
3,20210429,"[{""gamePk"":634330,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-29"",""t...","[{""home"":1,""gamePk"":634330,""gameDate"":""2021-04...","[{""home"":0,""teamId"":119,""gamePk"":634346,""gameD...","[{""transactionId"":480993,""playerId"":606965,""pl...","[{""season"":2021,""gameDate"":""2021-04-29"",""divis...",NaN,"[{""gamePk"":634346,""gameDate"":""2021-04-29"",""gam...",NaN,NaN
4,20210430,"[{""gamePk"":634287,""gameType"":""R"",""season"":2021...","[{""playerId"":405395,""gameDate"":""2021-04-30"",""t...","[{""home"":1,""gamePk"":634305,""gameDate"":""2021-04...","[{""home"":1,""teamId"":135,""gamePk"":634303,""gameD...",NaN,"[{""season"":2021,""gameDate"":""2021-04-30"",""divis...","[{""awardId"":""NLRRELMON"",""awardName"":""NL Reliev...","[{""gamePk"":634327,""gameDate"":""2021-04-30"",""gam...",NaN,NaN
